In [ ]:
import torch, os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].

# Apply necessary image transfromations here 

transform = transforms.Compose([#torchvision.transforms.RandomAffine(degrees=(-10, 10), translate=(0.1, 0.1), scale=(0.8, 1.2)),
                                #torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
                                #torchvision.transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])])
print(transform)

Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)


In [ ]:
train_data_dir = '/content/drive/My Drive/Datasets/3/train' # put path of training dataset
val_data_dir = '/content/drive/My Drive/Datasets/3/val' # put path of validation dataset
test_data_dir = '/content/drive/My Drive/Datasets/3/test' # put path of test dataset

trainset = torchvision.datasets.ImageFolder(root= train_data_dir, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=4)

valset = torchvision.datasets.ImageFolder(root= val_data_dir, transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=4,
                                         shuffle=False, num_workers=4)

testset = torchvision.datasets.ImageFolder(root= test_data_dir, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=4)

In [ ]:
########################################################################
# Define a Convolution Neural Network
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Copy the neural network from the Neural Networks section before and modify it to
# take 3-channel images (instead of 1-channel images as it was defined).

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
# <<<<<<<<<<<<<<<<<<<<< EDIT THE MODEL DEFINITION >>>>>>>>>>>>>>>>>>>>>>>>>>
# Try experimenting by changing the following:
# 1. number of feature maps in conv layer
# 2. Number of conv layers
# 3. Kernel size
# etc etc.,
   

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5)
        self.fc3 = nn.Linear(in_features=512, out_features=33)      # change out_features according to number of classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        global layer1
        layer1 = x
        x = self.pool(F.relu(self.conv2(x)))
        global layer2
        layer2 = x
        x = self.pool(F.relu(self.conv3(x)))
        global layer3
        layer3 = x
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fc3(x)
        return x

net = Net()

if torch.cuda.is_available():
  net = net.cuda()

net.load_state_dict(torch.load('/content/drive/My Drive/Datasets/Final_Model.pth', map_location = torch.device('cpu')))
net.eval()

Net(
  (conv1): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
  (fc3): Linear(in_features=512, out_features=33, bias=True)
)

In [ ]:
type(net.conv1.weight)

torch.nn.parameter.Parameter

In [ ]:
########################################################################
# Let us look at how the network performs on the test dataset.

def test(testloader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: %d %%' % (
                                    100 * correct / total))

In [ ]:
test(testloader, net)

In [ ]:
def plot_filters_single_channel_big(t):
    
    #setting the rows and columns
    nrows = t.shape[0]*t.shape[2]
    ncols = t.shape[1]*t.shape[3]
    
    
    npimg = np.array(t.numpy(), np.float32)
    npimg = npimg.transpose((0, 2, 1, 3))
    npimg = npimg.ravel().reshape(nrows, ncols)
    
    npimg = npimg.T
    
    fig, ax = plt.subplots(figsize=(ncols/10, nrows/200))    
    imgplot = sns.heatmap(npimg, xticklabels=False, yticklabels=False, cmap='gray', ax=ax, cbar=False)

In [ ]:
def plot_filters_single_channel(t):
    
    #kernels depth * number of kernels
    nplots = t.shape[0]*t.shape[1]
    ncols = 12
    
    nrows = 1 + nplots//ncols
    #convert tensor to numpy image
    npimg = np.array(t.numpy(), np.float32)
    
    count = 0
    fig = plt.figure(figsize=(ncols, nrows))
    
    #looping through all the kernels in each channel
    for i in range(t.shape[0]):
        for j in range(t.shape[1]):
            count += 1
            ax1 = fig.add_subplot(nrows, ncols, count)
            npimg = np.array(t[i, j].numpy(), np.float32)
            npimg = (npimg - np.mean(npimg)) / np.std(npimg)
            npimg = np.minimum(1, np.maximum(0, (npimg + 0.5)))
            ax1.imshow(npimg)
            ax1.set_title(str(i) + ',' + str(j))
            ax1.axis('off')
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
   
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_filters_multi_channel(t):
    
    #get the number of kernals
    num_kernels = t.shape[0]    
    
    #define number of columns for subplots
    num_cols = 12
    #rows = num of kernels
    num_rows = num_kernels
    
    #set the figure size
    fig = plt.figure(figsize=(num_cols,num_rows))
    
    #looping through all the kernels
    for i in range(t.shape[0]):
        ax1 = fig.add_subplot(num_rows,num_cols,i+1)
        
        #for each kernel, we convert the tensor to numpy 
        npimg = np.array(t[i].numpy(), np.float32)
        #standardize the numpy image
        npimg = (npimg - np.mean(npimg)) / np.std(npimg)
        npimg = np.minimum(1, np.maximum(0, (npimg + 0.5)))
        npimg = npimg.transpose((1, 2, 0))
        ax1.imshow(npimg)
        ax1.axis('off')
        ax1.set_title(str(i))
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        
    plt.savefig('myimage.png', dpi=100)    
    plt.tight_layout()
    plt.show()

In [ ]:
c1_tensor = net.conv1.weight.cpu().data

In [ ]:
plot_filters_multi_channel(c1_tensor)

In [ ]:
img = Image.open("/content/drive/My Drive/Datasets/3/test/frying_pan/n0340023100000844.jpg")
print(type(img))
trans1 = transforms.ToTensor()
tensor = trans1(img)
input = (tensor.unsqueeze(0))
output = net(input)  
print(torch.max(output[0]))
print(output[0])
print(torch.eq(torch.max(output[0]), output[0][10]))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
tensor(5.6470, grad_fn=<MaxBackward1>)
tensor([ 1.7186,  4.3766, -4.6335,  1.9029, -0.4405,  3.4578,  1.3772,  0.2593,
         2.4365, -1.5706,  5.6470,  0.9799, -3.1920, -1.5036, -2.8335,  3.1338,
        -0.3243, -3.4037,  3.1597,  5.2929,  3.1672, -3.0863, -0.9179, -1.6296,
         0.0882, -1.1678, -3.3321, -1.9664, -5.2096, -1.8843,  0.7884, -0.1084,
         0.0615], grad_fn=<SelectBackward>)
tensor(True)


In [ ]:
net(input)

In [ ]:
global layer1
global layer2
global layer3

In [ ]:
torch.max(layer1[0][0])

In [ ]:
list1 = [0,0,0,0,0]
info_list1 =[]

In [ ]:
image_set = "/content/drive/My Drive/Datasets/3/test/"
k=0
min = 0
for folder in os.listdir(image_set):
  for filename in os.listdir(image_set + folder):
    img_path = image_set+folder+'/'+filename
    #print(image_set+folder+'/'+filename)
    img = Image.open(img_path)
    trans = transforms.ToTensor()
    tensor = trans(img)
    input = tensor.unsqueeze(0)  #.cuda()
    #print("Output here")        #output = net(input)
    for i in range(layer1[0][0].shape[0]):
      for j in range(layer1[0][0].shape[1]):
        if layer1[0][0][i][j] > min:
          list1.append(layer1[0][0][i][j])
          list1.sort()
          for k in range(len(info_list1)):
            if info_list1[k][1] == min:
              info_list1.pop(k)
              break 
          list1.pop(0)
          min = list1[0]
          info_list1.append([img_path, layer1[0][0][i][j], i, j])

        

print(k)

In [ ]:
thing = [1,2,3]
thing.sort()
thing.pop(0)
print(thing)

[2, 3]


In [ ]:
thing.append("ahdh")
thing.append(4)
print(thing)

['ahdh', 4]


In [ ]:
columns = ['Image Path', 'Response', 'i', 'j']
df1 = pd.DataFrame(columns=columns)

In [ ]:
df1.loc[df1.shape[0]] = ['img2', 4,0,0]

In [ ]:
df1.idxmax()

TypeError: ignored

In [ ]:
responses = df1.iloc[:, 1]

In [ ]:
responses.idxmax()

2

In [ ]:
df1[['Response', 'i', 'j']] = df1[['Response', 'i', 'j']].apply(pd.to_numeric) 

In [ ]:
df1

,Image Path,Response,i,j
0,img,0,0,0
1,img,3,0,0
2,img2,4,0,0
